## Strumenti utilizzati
- tabula java
- cURL;
- scrape;
- xml2json;
- jq;
- Python (pandas, requests e urllib)

## Download della lista dei bollettini in PDF

In [117]:
%%bash
# setto cartella di lavoro

cartella=$(pwd)
# Faccio un po' di pulizia
# cancello i CSV temporanei
rm -R "$cartella"/*.csv > /dev/null 2>&1

# http://www.rapspa.it/temp/index_file/bollettini_qa/vedi_file.php?mensile=NO&anno=2017
# creo la lista dei bollettini dei PDF presenti sul sito (in questo caso quelli dell'anno 2017)
curl -s 'http://www.rapspa.it/temp/index_file/bollettini_qa/vedi_file.php?mensile=NO&anno=2017' | scrape -be '//tr/td/a' \
| xml2json > "$cartella/out.json"

## La prima versione dei dati è questa

In [118]:
! cat $(pwd)/out.json | jq . | head -20

{
  "html": {
    "body": {
      "a": [
        {
          "style": "color: green; font-weight: bold; ",
          "href": "#",
          "onclick": "window.open(\"bollettino_20170405.pdf\",\"\",\"\")",
          "$t": "Bollettino del 05/04/2017"
        },
        {
          "style": "color: green; font-weight: bold; ",
          "href": "#",
          "onclick": "window.open(\"bollettino_20170404.pdf\",\"\",\"\")",
          "$t": "Bollettino del 04/04/2017"
        },
        {
          "style": "color: green; font-weight: bold; ",
          "href": "#",
          "onclick": "window.open(\"bollettino_20170403.pdf\",\"\",\"\")",


## Faccio un po' di pulizia e converto in CSV

- rimuovo da `onclick` tutto quello che mi consente di mantenere solo il nome del PDF;
- rimuovo da `$t` tutto quello che mi consente di mantenere soltanto la data;

In [119]:
%%bash
cartella=$(pwd)
<"$cartella/out.json" sed 's/window.open(\\\"//g ; s/\\\",\\\"\\\",\\\"\\\")//g ; s/Bollettino del //g; s/$t/data/g; s/onclick/file/g' \
| jq '.html.body.a' | in2csv -f json --no-inference | csvcut -c 3,4 > "$cartella/out.txt"
rm "$cartella/out.json"

## Ecco il CSV "pulito"

In [120]:
! cat $(pwd)/out.txt | csvlook | head -8

|--------------------------+-------------|
|  file                    | data        |
|--------------------------+-------------|
|  bollettino_20170405.pdf | 05/04/2017  |
|  bollettino_20170404.pdf | 04/04/2017  |
|  bollettino_20170403.pdf | 03/04/2017  |
|  bollettino_20170402.pdf | 02/04/2017  |
|  bollettino_20170401.pdf | 01/04/2017  |


## Attivo la parte Python

In [121]:
# importo pandas per gestire i CSV
from pandas import *
# importo pandas e urllib per fare il download dei PDF
import requests
import urllib
import matplotlib.pyplot as plt

# imposto cartella di lavoro
cartella=%pwd

# creo un dataframe con l'elenco dei PDF giornalieri presenti sul sito della RAP
dataframe=read_csv('out.txt')
dataframe.head(3)

,file,data
0,bollettino_20170405.pdf,05/04/2017
1,bollettino_20170404.pdf,04/04/2017
2,bollettino_20170403.pdf,03/04/2017


In [122]:
# converto la colonna data da testuale a data
dataframe['data'] = to_datetime(dataframe['data'])
dataframe.head(3)

,file,data
0,bollettino_20170405.pdf,2017-05-04
1,bollettino_20170404.pdf,2017-04-04
2,bollettino_20170403.pdf,2017-03-04


In [123]:
# imposto come colonna indice quella delle date
dataframe.index = dataframe['data']
dataframe.head(3)

,file,data
data,,
2017-05-04,bollettino_20170405.pdf,2017-05-04
2017-04-04,bollettino_20170404.pdf,2017-04-04
2017-03-04,bollettino_20170403.pdf,2017-03-04


In [124]:
# URL radice dei bolletini giornalieri della RAP
dowloadPath="http://www.rapspa.it/temp/index_file/bollettini_qa/"

# creo la lista dei PDF giornalieri già scaricati
lista=! echo $(for f in $(pwd)/export/*.pdf; do echo $f; done) | sed "s/ /,/g ; s/,/\n/g  ; s|$(pwd)/export/||g"

# creo file per avere un log degli eventuali problemi di download
logf = open("download.log", "w")


# scarico solo i PDF non presenti già in archivio
for i in dataframe['file']:
    if i not in lista:
        filePDF = urllib.URLopener()
        try:
            filePDF.retrieve(dowloadPath+i, cartella + '/' + i)
        except Exception as e:
            logf.write("Failed to download {0}: {1}\n".format(str(i), str(e)))
        finally:
            pass
        
logf.close()

In [125]:
#Mi faccio restituire i PDF non scaricabili dal sito
!cat download.log

Failed to download bollettino_20170321.pdf: ('http error', 404, 'Not Found', <httplib.HTTPMessage instance at 0x7eff6cc99290>)


## Conversione dei PDF in CSV

Per farlo uso [Tabula](http://tabula.technology/). Tutti i bollettini in PDF hanno la stessa formattazione, e la tabella con i dati è sempre circoscritta in un poligono con le stesse coordinate; dirò a Tabula di agire sull'area inclusa in questo poligono.

In [126]:
%%bash
cartella=$(pwd)

rm "$cartella/out.txt"

# estraggo con Tabula i dati dal PDF
for f in "$cartella"/*.pdf; do
    filename=$(basename "$f")
    extension="${filename##*.}"
    filename="${filename%.*}"
    java -jar tabula-java.jar -p all  -a 130.528,75.491,328.366,531.409 -o $filename.csv $filename.pdf -n
done

## Elaboro i CSV

In [127]:
%%bash
cartella=$(pwd)

# inserisco una riga di intestazione nei CSV creati
for f in "$cartella"/*.csv; do
    filename=$(basename "$f")
    extension="${filename##*.}"
    filename="${filename%.*}"
    sed -i '1s/^/Stazione_di_misura,S02media_24_h,COmedia_1_h_max,COmedia_8_ore_max,NO2,O3media_1_h_max,O3media_8_ore_max,PM10\n/' "$filename".csv
done

In [128]:
import glob,os
cwd = os.getcwd()
os.chdir(cwd)

if not os.path.exists(cwd + '/out'):
    os.makedirs(cwd + '/out')

# Leggi tutti i file CSV presenti nella radice    
for file in glob.glob("*.csv"):
    #print(file)
    fileCSV=read_csv(file,converters={'Stazione_di_misura': str,'S02media_24_h': str,'COmedia_1_h_max': str,'COmedia_8_ore_max': str,'NO2': str,'O3media_1_h_max': str,'O3media_8_ore_max': str,'PM10': str},engine='python')
    # aggiungo la colonna con la data    
    fileCSV['data']=file
    
    # rimuovo le righe in cui la prima colonna, quelle delle stazioni è NaN
    fileCSV = fileCSV[fileCSV.Stazione_di_misura.notnull()]
    
    # Inserisco nelle righe vuote la stringa `SnA`, Stazione non Abilitata
    fileCSV=fileCSV.fillna(value='SnA')
    
    # Trasformo le celle `nd` in NaN
    fileCSV = fileCSV.replace('nd',np.NaN)
    
    # rimuovo ',' e sostituisco con '.'
    fileCSV = fileCSV.apply(lambda x: x.str.replace(',', '.'))
    fileCSV['day']=''
    fileCSV['month']=''
    fileCSV['year']=''
    fileCSV['NotaPM10']=''
    # salvo il file
    fileCSV.to_csv(cwd + '/out/' + file,encoding='utf-8',index=False,decimal='.')

In [129]:
%%bash
cartella=$(pwd)

if [ ! -f "$cartella"/export/dati.csv ]
then
    echo "File not found!"
else
    cat "$cartella"/export/dati.csv | sed 's/\-//g' > "$cartella"/out/00.csv
fi

In [130]:
%%bash
# unisco i CSV e pulisco la colonna data rimuovendo la stringa "bollettino_"
cartella=$(pwd)
controllo=$(ls -1 "$cartella/out/"*.csv | wc -l)
if (( $controllo > 1 )); then
    csvstack ./out/*.csv | sed "s/bollettino_//g ; s/.csv//g ; s/Unità d Italia '/Unità d'Italia/g ; s/Unità d 'Italia/Unità d'Italia/g" > dati.txt
else
    echo "non ci sono file CSV da unire"
fi

In [131]:
%%bash
# Faccio un po' di pulizia
cartella=$(pwd)

# cancello i CSV temporanei
rm -R "$cartella"/*.csv

# creo la cartella export, se non esiste
mkdir -p "$cartella"/export

# muovo i PDF con i bollettini giornalieri in export
mv "$cartella"/*.pdf "$cartella"/export

# rimuovo i CSV temporaneri dalla radice
rm -R "$cartella"/out/*.csv

In [132]:
if os.stat('dati.txt').st_size != 0:
    datiDF=read_csv('dati.txt')
    datiDF['data'] = to_datetime(datiDF['data'], format='%Y%m%d')
    datiDF.index = datiDF['data']
    datiDF['day']=datiDF.index.day
    datiDF['month']=datiDF.index.month
    datiDF['year']=datiDF.index.year
    datiDF['PM10'] = datiDF['PM10'].astype(str)
    # creo una colonna per dare conto della nota alle volte presente sui valori di PM10
    if not os.path.isfile(cwd + '/export/dati.csv'):
        datiDF['NotaPM10']=''
    datiDF.loc[datiDF['PM10'].str.contains('\*', na=False),'NotaPM10'] = 'x'
    datiDF.replace({' ?\* ?':''}, regex=True,inplace=True)
    datiDF.replace({'G$':''}, regex=True,inplace=True)
        # se la cartella export non esiste, creala
    if not os.path.exists(cwd + '/export'):
        os.makedirs(cwd + '/export')
    # faccio il merge con le stazioni
    datiDF = datiDF.drop('longitude', 1)
    datiDF = datiDF.drop('latitude', 1)
    stazioni=read_csv("./data/RAPStazioniAriaMonitoraggio.csv")
    stazioni=stazioni[['stazione','longitude','latitude']]
    datiDF=merge(datiDF, stazioni, how='left',left_on=['Stazione_di_misura'], right_on=['stazione'])
    datiDF=datiDF.drop(['stazione'], axis=1)
    # salva il file nella cartella export
    datiDF.replace("nan","",inplace=True)
    datiDF['PM10'] = to_numeric(datiDF['PM10'])
    datiDF.drop_duplicates(inplace=True)
    datiDF.sort_values(['data','Stazione_di_misura'], ascending=[True,True],inplace=True)
    datiDF.to_csv(cwd + '/export/dati.csv',encoding='utf-8',index=False,decimal='.')

# Creo la versione tidy

In [134]:
datiDF.index = datiDF['data']
datiDF.head(3)

,Stazione_di_misura,S02media_24_h,COmedia_1_h_max,COmedia_8_ore_max,NO2,O3media_1_h_max,O3media_8_ore_max,PM10,data,day,month,year,NotaPM10,longitude,latitude
data,,,,,,,,,,,,,,,
2016-01-01,Belgio,SnA,NaN,NaN,,SnA,SnA,NaN,2016-01-01,1,1,2016,NaN,13.331566,38.151567
2016-01-01,Boccadifalco,1,0.2,0.1,67,66,62,23.0,2016-01-01,1,1,2016,NaN,13.302383,38.120513
2016-01-01,CEP,2,NaN,NaN,,SnA,SnA,57.0,2016-01-01,1,1,2016,NaN,13.308882,38.135752


## creo una copia del dataframe

Questa copia verrà sottosta a procedura di tidy

In [135]:
datiDFdue=datiDF[['Stazione_di_misura', 'data','S02media_24_h', 'COmedia_1_h_max', 'COmedia_8_ore_max', 'NO2', 'O3media_1_h_max', 'O3media_8_ore_max', 'PM10','NotaPM10', 'longitude', 'latitude']]
datiDFdue.head(3)

,Stazione_di_misura,data,S02media_24_h,COmedia_1_h_max,COmedia_8_ore_max,NO2,O3media_1_h_max,O3media_8_ore_max,PM10,NotaPM10,longitude,latitude
data,,,,,,,,,,,,
2016-01-01,Belgio,2016-01-01,SnA,NaN,NaN,,SnA,SnA,NaN,NaN,13.331566,38.151567
2016-01-01,Boccadifalco,2016-01-01,1,0.2,0.1,67,66,62,23.0,NaN,13.302383,38.120513
2016-01-01,CEP,2016-01-01,2,NaN,NaN,,SnA,SnA,57.0,NaN,13.308882,38.135752


## Eseguo il tidy

Riporto su due sole colonne il tipo di inquinante e il valore associato.

In [136]:
tidyPA = pandas.melt(datiDFdue, 
                     id_vars=['Stazione_di_misura','data','NotaPM10', 'longitude', 'latitude'],
                     var_name='inquinante', 
                     value_name='valore')

## Creo indice, riordino le colonne e trasformo tutto in stringa

In [137]:
tidyPA.index = tidyPA['data']
tidyPA.sort_index(inplace=True)
tidyPA.sort_values(['data','Stazione_di_misura','inquinante'], ascending=[True,True,True],inplace=True)
tidyPA.astype('str',inplace=True)
tidyPA.replace({'':'NaN'}, regex=True,inplace=True)
tidyPA.head(10)

,Stazione_di_misura,data,NotaPM10,longitude,latitude,inquinante,valore
data,,,,,,,
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,COmedia_1_h_max,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,COmedia_8_ore_max,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,NO2,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,O3media_1_h_max,SnA
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,O3media_8_ore_max,SnA
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,PM10,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,S02media_24_h,SnA
2016-01-01,Boccadifalco,2016-01-01,NaN,13.302383,38.120513,COmedia_1_h_max,0.2
2016-01-01,Boccadifalco,2016-01-01,NaN,13.302383,38.120513,COmedia_8_ore_max,0.1


In [138]:
# rimuovo eventual duplicati
tidyPA.drop_duplicates(inplace=True)
tidyPA.head(10)

,Stazione_di_misura,data,NotaPM10,longitude,latitude,inquinante,valore
data,,,,,,,
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,COmedia_1_h_max,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,COmedia_8_ore_max,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,NO2,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,O3media_1_h_max,SnA
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,O3media_8_ore_max,SnA
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,PM10,NaN
2016-01-01,Belgio,2016-01-01,NaN,13.331566,38.151567,S02media_24_h,SnA
2016-01-01,Boccadifalco,2016-01-01,NaN,13.302383,38.120513,COmedia_1_h_max,0.2
2016-01-01,Boccadifalco,2016-01-01,NaN,13.302383,38.120513,COmedia_8_ore_max,0.1


# Pulizia colonna valori e crazione dei tipi di valore

L'obiettivo è quello di lasciare nella colonna valore solo i dati numerici e nella colonna tipo impostare questa classificazione:

- `NaN`, per i casi di "valore non disponibile";
- `valore`, per i casi in cui è stato assegnato un valore numerico;
- `<`, per i casi di "valore al di sotto del limite di rilevabilità";
- `SnA`, per i casi in cui si tratta di "stazione non abilitata alla misura".

I recordi classificati come `SnA` vengono rimossi dal dataframe.

In [139]:
tidyPA=tidyPA.astype('str')
tidyPA.replace({'nan':'NaN'}, regex=True,inplace=True)
tidyPA['tipo'] = np.where(tidyPA['valore'].str.contains('^[a-zA-Z]'), tidyPA['valore'],'')
tidyPA['tipo'] = np.where(tidyPA['valore'].str.contains('^<'), tidyPA['valore'],tidyPA['tipo'])
tidyPA['tipo'] = np.where(tidyPA['valore'].str.contains('^[0-9]'), 'valore',tidyPA['tipo'])
tidyPA['valore'] = np.where(tidyPA['valore'].str.contains('^[a-zA-Z]'), '',tidyPA['valore'])
tidyPA['valore'] = np.where(tidyPA['valore'].str.contains('^<'), '',tidyPA['valore'])
tidyPA = tidyPA[tidyPA.tipo != 'SnA']
tidyPA.replace({'NotaPM10': {'NaN': ''}},inplace=True)
tidyPA.head(20)

,Stazione_di_misura,data,NotaPM10,longitude,latitude,inquinante,valore,tipo
data,,,,,,,,
2016-01-01,Belgio,2016-01-01,,13.3315658569,38.1515674216,COmedia_1_h_max,,NaN
2016-01-01,Belgio,2016-01-01,,13.3315658569,38.1515674216,COmedia_8_ore_max,,NaN
2016-01-01,Belgio,2016-01-01,,13.3315658569,38.1515674216,NO2,,NaN
2016-01-01,Belgio,2016-01-01,,13.3315658569,38.1515674216,PM10,,NaN
2016-01-01,Boccadifalco,2016-01-01,,13.3023834229,38.1205128923,COmedia_1_h_max,0.2,valore
2016-01-01,Boccadifalco,2016-01-01,,13.3023834229,38.1205128923,COmedia_8_ore_max,0.1,valore
2016-01-01,Boccadifalco,2016-01-01,,13.3023834229,38.1205128923,NO2,67.0,valore
2016-01-01,Boccadifalco,2016-01-01,,13.3023834229,38.1205128923,O3media_1_h_max,66,valore
2016-01-01,Boccadifalco,2016-01-01,,13.3023834229,38.1205128923,O3media_8_ore_max,62,valore


In [140]:
# Associo altre metainformazioni ai dati

In [141]:
listainquinanti=[{"inquinante":"COmedia_1_h_max","units":"mg/m3","periodo":"media_1_ora_ max","nome":"CO"},
                 {"inquinante":"COmedia_8_ore_max","units":"mg/m3","periodo":"media_8_ore_max","nome":"CO"},
            {"inquinante":"NO2","units":"µg/m3","periodo":"media_1_ora_max","nome":"NO2"},
                 {"inquinante":"O3media_1_h_max","units":"µg/m3","periodo":"media_1_ora_max","nome":"O3"},
           {"inquinante":"O3media_8_ore_max","units":"µg/m3","periodo":"media_8_ore_max","nome":"O3"},
                 {"inquinante":"PM10","units":"µg/m3","periodo":"media_24_h","nome":"PM10"},
           {"inquinante":"S02media_24_h","units":"µg/m3","periodo":"media_24_h","nome":"S02"}]
inquinanti=pandas.DataFrame(listainquinanti)
#inquinanti=inquinanti[['inquinante','periodo','units']]
#tidyPA.join(inquinanti, on="inquinante")
tidyPAExport=tidyPA.merge(inquinanti, on='inquinante', how='left')
tidyPAExport = tidyPAExport.drop('inquinante', 1)
tidyPAExport=tidyPAExport.rename(columns = {'nome':'inquinante','nome':'inquinante'})
tidyPAExport=tidyPAExport[['data','Stazione_di_misura', 'inquinante', 'periodo','valore','units','tipo','NotaPM10', 'longitude', 
                          'latitude']]
tidyPAExport.head(5)

,data,Stazione_di_misura,inquinante,periodo,valore,units,tipo,NotaPM10,longitude,latitude
0,2016-01-01,Belgio,CO,media_1_ora_ max,,mg/m3,NaN,,13.3315658569,38.1515674216
1,2016-01-01,Belgio,CO,media_8_ore_max,,mg/m3,NaN,,13.3315658569,38.1515674216
2,2016-01-01,Belgio,NO2,media_1_ora_max,,µg/m3,NaN,,13.3315658569,38.1515674216
3,2016-01-01,Belgio,PM10,media_24_h,,µg/m3,NaN,,13.3315658569,38.1515674216
4,2016-01-01,Boccadifalco,CO,media_1_ora_ max,0.2,mg/m3,valore,,13.3023834229,38.1205128923


In [142]:
tidyPAExport.to_csv(cwd + '/export/datiTidy.csv',encoding='utf-8',index=False,decimal='.')